<a href="https://colab.research.google.com/github/MurtazaG786/LangGraph/blob/main/BloggerAi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q langgraph langchain langchain-community transformers accelerate torch


In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto"
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cuda:0


  **Blogger Ai**

In [14]:
from __future__ import annotations
import operator
from typing import TypedDict,List,Annotated
from pydantic import BaseModel,Field
from langgraph.graph import StateGraph,START,END
from langgraph.types import Send
from langchain_core.messages import HumanMessage,SystemMessage

In [15]:
class Task(BaseModel):
  id:int
  title:str
  desc:str=Field(...,description="What to cover")

In [16]:
class Plan(BaseModel):
  blog_title:str
  tasks:List[Task]

In [18]:
class State(TypedDict):
  topic:str
  plan:Plan
  sections:Annotated[List[str],operator.add]
  final:str

In [17]:
from langchain_community.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=pipe)

In [19]:
#node-1 this one will create a plan to write down the blog

def orchestrator(state:State):
  plan=llm.with_structured_output(Plan).invoke(
      [
          SystemMessage(
              content=(
                  """You are a senior content strategist and blog editor.

Your task is to generate a clear, structured plan for a high-quality blog post based on the given topic.

Rules:
- Do NOT write the full blog content.
- Do NOT include filler text or generic advice.
- Focus only on planning and structure.

For the given topic, produce:
1. A compelling blog title (SEO-friendly).
2. A one-paragraph introduction outline (key points only).
3. A detailed section outline using H2 and H3 headings.
4. Bullet points under each section describing what should be covered.
5. Suggestions for examples, analogies, or case studies where relevant.
6. A strong conclusion outline (key takeaways only).

Style guidelines:
- Logical and well-organized.
- Suitable for an 800–1200 word blog.
- Clear, practical, and reader-focused.
- No emojis, no markdown formatting beyond headings.

Return ONLY the plan. Do not generate the final blog text."""
              )
          ),
          HumanMessage(content=f"Topic : {state['topic']}"),
      ]
  )
  return {"plan":plan}